# Инициализация

In [ ]:
# # catboost из requiements на новых версия Python не будет нормально работать, поэтому:
# ! python3 --version
# # ! brew install python@3.11
# # ! brew install openblas
# ! python3.11 --version
# # ! curl -O https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-arm64.sh
# # ! bash Miniconda3-latest-MacOSX-arm64.sh
# ! conda --version
# ! conda create -n myenv python=3.11
# ! conda init
# ! conda activate myenv
# ! conda install -c conda-forge lightfm
# ! pip install -r requirements.txt

In [3]:
! wget https://storage.yandexcloud.net/mle-data/ym/tracks.parquet
! wget https://storage.yandexcloud.net/mle-data/ym/catalog_names.parquet
! wget https://storage.yandexcloud.net/mle-data/ym/interactions.parquet

--2025-10-25 22:36:01--  https://storage.yandexcloud.net/mle-data/ym/tracks.parquet
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18938128 (18M) [application/x-www-form-urlencoded]
Saving to: ‘tracks.parquet’

tracks.parquet      100%[===================>]  18.06M  12.9MB/s    in 1.4s    

2025-10-25 22:36:03 (12.9 MB/s) - ‘tracks.parquet’ saved [18938128/18938128]

--2025-10-25 22:36:03--  https://storage.yandexcloud.net/mle-data/ym/catalog_names.parquet
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36146495 (34M) [application/x-www-form-urlencoded]
Saving to: ‘catalog_names.parquet’

catalog_names.parqu 100%[========

Загружаем библиотеки необходимые для выполнения кода ноутбука.

In [1]:
import pandas as pd
import numpy as np
import polars as pl
import os
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
from dotenv import load_dotenv
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import scipy
import sklearn.preprocessing
from implicit.als import AlternatingLeastSquares
from scipy.sparse import csr_matrix
import pickle
from collections import Counter, defaultdict
from joblib import Parallel, delayed
from tqdm import tqdm
import math
from catboost import CatBoostClassifier, Pool

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# === ЭТАП 1 ===

# Загрузка первичных данных

Загружаем первичные данные из файлов:
- tracks.parquet
- catalog_names.parquet
- interactions.parquet

In [2]:
%%time
tracks = pd.read_parquet('tracks.parquet')
catalog_names = pd.read_parquet('catalog_names.parquet')
interactions = pd.read_parquet('interactions.parquet')

CPU times: user 330 ms, sys: 56.5 ms, total: 387 ms
Wall time: 337 ms


In [5]:
tracks.head(3)
catalog_names.head(3)
interactions.head(3)

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]


,id,type,name
0,3,album,Taller Children
1,12,album,Wild Young Hearts
2,13,album,Lonesome Crow


,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19
2,0,590262,3,2022-07-21


# Обзор данных

Проверяем данные, есть ли с ними явные проблемы.

In [11]:
tracks.describe()
catalog_names.describe()
interactions.describe()

,track_id
count,1.000000e+06
mean,3.685121e+07
std,2.679771e+07
min,2.600000e+01
25%,1.543088e+07
50%,3.455047e+07
75%,5.692557e+07
max,1.015218e+08


,id
count,1.812471e+06
mean,2.321647e+07
std,2.526044e+07
min,0.000000e+00
25%,3.480524e+06
50%,1.211436e+07
75%,3.773817e+07
max,1.015218e+08


,user_id,track_id,track_seq,started_at
count,2.226299e+08,2.226299e+08,2.226299e+08,222629898
mean,6.875767e+05,3.653622e+07,4.621403e+02,2022-08-29 16:39:44.541336320
min,0.000000e+00,2.600000e+01,1.000000e+00,2022-01-01 00:00:00
25%,3.433710e+05,1.480849e+07,5.600000e+01,2022-07-02 00:00:00
50%,6.879730e+05,3.552474e+07,1.810000e+02,2022-09-15 00:00:00
75%,1.031127e+06,5.651137e+07,5.060000e+02,2022-11-09 00:00:00
max,1.374582e+06,1.015218e+08,1.663700e+04,2022-12-31 00:00:00
std,3.969033e+05,2.661782e+07,8.257312e+02,NaN


In [12]:
tracks.info()
catalog_names.info()
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   track_id  1000000 non-null  int64 
 1   albums    1000000 non-null  object
 2   artists   1000000 non-null  object
 3   genres    1000000 non-null  object
dtypes: int64(1), object(3)
memory usage: 30.5+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1812471 entries, 0 to 1812470
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   id      int64 
 1   type    object
 2   name    object
dtypes: int64(1), object(2)
memory usage: 41.5+ MB
<class 'pandas.core.frame.DataFrame'>
Index: 222629898 entries, 0 to 291
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   user_id     int32         
 1   track_id    int32         
 2   track_seq   int16         
 3   started_at  datetime64[ns]
dtypes: datetime64[ns](1), int16(1), i

In [13]:
interactions.agg({'user_id':['min','max']})

,user_id
min,0
max,1374582


In [14]:
user_ids = list(interactions['user_id'].unique())
start_id = 1000000
id_mapping = {old_id: new_id for new_id, old_id in enumerate(user_ids, start=start_id)}
interactions_v1 = interactions.copy()
interactions_v1['user_id'] = interactions_v1['user_id'].map(id_mapping)
interactions_v1

,user_id,track_id,track_seq,started_at
0,1000000,99262,1,2022-07-17
1,1000000,589498,2,2022-07-19
2,1000000,590262,3,2022-07-21
3,1000000,590303,4,2022-07-22
4,1000000,590692,5,2022-07-22
...,...,...,...,...
287,2373220,95514767,288,2022-12-29
288,2373220,97220301,289,2022-12-29
289,2373220,97241171,290,2022-12-29
290,2373220,98542641,291,2022-12-29


In [15]:
interactions_v1.agg({'user_id':['min','max']})

,user_id
min,1000000
max,2373220


In [16]:
len(tracks)

1000000

In [17]:
tracks[tracks['albums'].apply(len)==0]

,track_id,albums,artists,genres
310821,20200372,[],[],[]
310826,20200380,[],[],[]
312469,20305116,[],[],[]
312474,20305121,[],[],[]
320353,20756854,[],[],[]
326588,21196099,[],[],[]
326592,21196103,[],[],[]
326594,21196105,[],[],[]
326596,21196107,[],[],[]
326598,21196109,[],[],[]


In [18]:
tracks[tracks['artists'].apply(len)==0]

,track_id,albums,artists,genres
146031,3599314,[389925],[],"[68, 86]"
146163,3599591,[389944],[],"[68, 86]"
177544,4790215,[533785],[],"[68, 86]"
218922,10063296,[1083742],[],"[11, 20]"
231455,12122918,[1315834],[],[68]
...,...,...,...,...
999868,101201589,[12649219],[],"[62, 77]"
999876,101219571,[9444321],[],"[62, 83, 77]"
999901,101243616,[11700332],[],"[62, 93]"
999920,101292599,[9331881],[],"[62, 90, 77, 84]"


In [19]:
tracks[tracks['genres'].apply(len)==0]

,track_id,albums,artists,genres
490,2520,"[249, 163801, 163802, 324616, 17542430, 189498...",[2259],[]
3069,16776,"[1479, 5319, 6093, 8467, 10641, 90867, 163801,...",[978],[]
3077,16801,"[1479, 163801, 9287549, 9377183, 9424994, 1201...",[10520],[]
4192,23752,"[2001, 3722, 4973, 6104, 6558, 23866, 37052, 7...",[13373],[]
6645,38012,"[3147, 60099, 530836, 1115289, 2490724, 818446...",[16758],[]
...,...,...,...,...
997667,99412992,[20764551],[10029101],[]
999192,100497082,[21098340],[15936615],[]
999703,100923319,[21233460],[5271861],[]
999704,100923322,[21233460],[5271861],[]


In [20]:
nonvalid_tracks=\
list(tracks[tracks['albums'].apply(len)==0]['track_id'].unique())+\
list(tracks[tracks['artists'].apply(len)==0]['track_id'].unique())+\
list(tracks[tracks['genres'].apply(len)==0]['track_id'].unique())
nonvalid_tracks[:5]

[20200372, 20200380, 20305116, 20305121, 20756854]

In [21]:
tracks_v1 = tracks.query('~track_id.isin(@nonvalid_tracks)')
tracks_v1

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 24...",[84],[11]
...,...,...,...,...
999995,101478482,[21399811],[5540395],"[3, 75]"
999996,101490148,[21403052],[9078726],"[11, 20]"
999997,101493057,[21403883],[11865715],"[44, 75]"
999998,101495927,[21404975],[4462686],"[3, 75]"


# Выводы

Приведём выводы по первому знакомству с данными:
- есть ли с данными явные проблемы,
- какие корректирующие действия (в целом) были предприняты.

In [22]:
# идентификаторы user_id приведены к 7-значному формату
interactions_v1.head()

,user_id,track_id,track_seq,started_at
0,1000000,99262,1,2022-07-17
1,1000000,589498,2,2022-07-19
2,1000000,590262,3,2022-07-21
3,1000000,590303,4,2022-07-22
4,1000000,590692,5,2022-07-22


In [23]:
# присутствуют строки с отсутствующими исполнителями, альбомами, жанрами
# просто удаляем записи с пропусками
len(tracks), len(tracks_v1)

(1000000, 980977)

# === ЭТАП 2 ===

# EDA

Распределение количества прослушанных треков.

In [24]:
%%time
interactions_v1_pl = pl.from_pandas(interactions_v1)
interactions_v1_pl

CPU times: user 192 ms, sys: 450 ms, total: 642 ms
Wall time: 792 ms


user_id,track_id,track_seq,started_at
i64,i32,i16,datetime[ns]
1000000,99262,1,2022-07-17 00:00:00
1000000,589498,2,2022-07-19 00:00:00
1000000,590262,3,2022-07-21 00:00:00
1000000,590303,4,2022-07-22 00:00:00
1000000,590692,5,2022-07-22 00:00:00
…,…,…,…
2373220,95514767,288,2022-12-29 00:00:00
2373220,97220301,289,2022-12-29 00:00:00
2373220,97241171,290,2022-12-29 00:00:00


In [25]:
%%time
tracks_dist=\
interactions_v1_pl.group_by('user_id').agg(pl.col('track_id').n_unique().alias('listenings')).sort('listenings',descending=True)
tracks_dist

CPU times: user 5.03 s, sys: 1.95 s, total: 6.98 s
Wall time: 1.41 s


user_id,listenings
i64,u32
1398587,16637
2199698,15392
1590301,15136
1730511,13223
1339385,12768
…,…
2010329,1
2095280,1
1166447,1


In [26]:
n_bins = 10

tracks_dist = tracks_dist.with_columns([
    # Compute rank of nunique, scale to bin id (0 to 9), cast to int
    ( (pl.col("listenings").rank("average") - 1) / tracks_dist.height * n_bins )
        .cast(int)
        .alias("listenings_q")
])

print(tracks_dist)

shape: (1_373_221, 3)
┌─────────┬────────────┬──────────────┐
│ user_id ┆ listenings ┆ listenings_q │
│ ---     ┆ ---        ┆ ---          │
│ i64     ┆ u32        ┆ i64          │
╞═════════╪════════════╪══════════════╡
│ 1398587 ┆ 16637      ┆ 9            │
│ 2199698 ┆ 15392      ┆ 9            │
│ 1590301 ┆ 15136      ┆ 9            │
│ 1730511 ┆ 13223      ┆ 9            │
│ 1339385 ┆ 12768      ┆ 9            │
│ …       ┆ …          ┆ …            │
│ 2010329 ┆ 1          ┆ 0            │
│ 2095280 ┆ 1          ┆ 0            │
│ 1166447 ┆ 1          ┆ 0            │
│ 1017806 ┆ 1          ┆ 0            │
│ 2163445 ┆ 1          ┆ 0            │
└─────────┴────────────┴──────────────┘


In [27]:
tracks_dist.group_by('listenings_q').agg(pl.col('user_id').n_unique().alias('listeners')).sort('listenings_q',descending=True)

listenings_q,listeners
i64,u32
9,137445
8,137581
7,136828
6,139071
5,132735
4,136567
3,145596
2,126235
1,156961


In [28]:
tracks_dist_min_max=\
tracks_dist.group_by("listenings_q").agg([
    pl.col("listenings").min().alias("min_listened_tracks"),
    pl.col("listenings").max().alias("max_listened_tracks"),
]).sort('listenings_q',descending=True)
tracks_dist_min_max

listenings_q,min_listened_tracks,max_listened_tracks
i64,u32,u32
9,389,16637
8,200,388
7,122,199
6,81,121
5,56,80
4,39,55
3,27,38
2,20,26
1,14,19


In [ ]:
# 10% самых активных пользователей слушают от 389 до 16637 треков
# 10% самых неактивных – от 1 до 13

Наиболее популярные треки

In [148]:
%%time
popular_tracks=\
interactions_v1.groupby('track_id')['user_id'].count().reset_index()\
.rename(columns={'user_id':'listenings'}).sort_values(by='listenings',ascending=False)\
.merge(catalog_names.query('type=="track"'), left_on='track_id', right_on='id').drop(columns=['type','id']).rename(columns={'name':'track'})
popular_tracks=\
popular_tracks.merge(tracks.explode('artists')[['track_id','artists']].drop_duplicates()\
.merge(catalog_names.query('type=="artist"'), left_on='artists', right_on='id').drop(columns=['type','id','artists']),on='track_id')\
.rename(columns={'name':'artist'})\
[['track_id','artist','track','listenings']].drop_duplicates()\
.groupby(['track_id','track','listenings']).aggregate(lambda tdf: tdf.unique().tolist())\
.sort_values(by='listenings',ascending=False).reset_index()[['artist','track','listenings']]
popular_tracks.head(10)

CPU times: user 15.3 s, sys: 674 ms, total: 16 s
Wall time: 16 s


,artist,track,listenings
0,[Nirvana],Smells Like Teen Spirit,111062
1,[Imagine Dragons],Believer,106921
2,[Linkin Park],Numb,101924
3,"[Рем Дигга, Miyagi & Эндшпиль]",I Got Love,99490
4,[Dabro],Юность,86670
5,[KALEO],Way Down We Go,86246
6,[Ed Sheeran],Shape of You,85886
7,"[Fleurie, Tommee Profitt, Mellen Gi]",In The End,85244
8,[Sting],Shape Of My Heart,85042
9,[Zivert],Life,84748


Наиболее популярные жанры

In [149]:
tracks.head(2)
catalog_names.head(2)
interactions.head(2)

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"


,id,type,name
0,3,album,Taller Children
1,12,album,Wild Young Hearts


,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19


In [178]:
popular_genres=\
tracks[['track_id','genres']].explode('genres').merge(catalog_names.query('type=="genre"'),left_on='genres',right_on='id')\
[['track_id','name']].rename(columns={'name':'genre'})\
.merge(interactions.reset_index().groupby('track_id')['index'].count().reset_index().rename(columns={'index':'interactions'}),
on='track_id').sort_values(by='interactions',ascending=False)\
.groupby('genre')['interactions'].sum().reset_index().sort_values(by='interactions',ascending=False)
popular_genres.head(10)

,genre,interactions
91,pop,55578312
97,rap,37799821
1,allrock,31092013
110,ruspop,26626241
111,rusrap,25303695
39,electronics,20120981
32,dance,16291557
112,rusrock,13166147
105,rock,12772644
79,metal,12437375


Треки, которые никто не прослушал

In [157]:
interactions.head()

,user_id,track_id,track_seq,started_at
0,0,99262,1,2022-07-17
1,0,589498,2,2022-07-19
2,0,590262,3,2022-07-21
3,0,590303,4,2022-07-22
4,0,590692,5,2022-07-22


In [162]:
%%time
tracks[['track_id']].drop_duplicates().rename(columns={'track_id':'existing_track_id'})\
.merge(interactions.reset_index().groupby('track_id')['index'].count().reset_index().rename(columns={'index':'interactions'}),
left_on='existing_track_id',right_on='track_id',how='left').sort_values(by='interactions',ascending=True)

CPU times: user 2.42 s, sys: 2.45 s, total: 4.87 s
Wall time: 6.21 s


,existing_track_id,track_id,interactions
0,26,26,5
63299,607061,607061,5
798525,62451520,62451520,5
960125,87921040,87921040,5
960129,87921117,87921117,5
...,...,...,...
829320,65851540,65851540,86670
512157,35505245,35505245,99490
26665,178529,178529,101924
483876,33311009,33311009,106921


In [164]:
tracks['track_id'].nunique()
interactions['track_id'].nunique()

1000000

1000000

In [ ]:
# Выглядит так, как будто у нас здесь данные только прослушанных треках

# Преобразование данных

Преобразуем данные в формат, более пригодный для дальнейшего использования в расчётах рекомендаций.

In [179]:
! pwd

/Users/Konstantin/Study/Practicum – ML Engineering/sprint_4/mle-project-sprint-4-v001


In [29]:
items = tracks_v1.copy()
events = interactions_v1.copy()

In [30]:
%%time
items.to_parquet('items.parquet')
events.to_parquet('events.parquet')

CPU times: user 12.3 s, sys: 2 s, total: 14.3 s
Wall time: 16.8 s


# Сохранение данных

Сохраним данные в двух файлах в персональном S3-бакете по пути `recsys/data/`:
- `items.parquet` — все данные о музыкальных треках,
- `events.parquet` — все данные о взаимодействиях.

In [188]:
! export $(grep -v '^#' .env | xargs)

In [31]:
! ls

Miniconda3-latest-MacOSX-arm64.sh items.parquet
README.md                         myenv
catalog_names.parquet             recommendations.ipynb
env_recsys_start                  requirements.txt
events.parquet                    tracks.parquet
interactions.parquet


In [24]:
bucket_name = os.getenv('S3_BUCKET_NAME')
aws_key = os.getenv('AWS_ACCESS_KEY_ID')
aws_secret = os.getenv('AWS_SECRET_ACCESS_KEY')
endpoint_url = 'https://storage.yandexcloud.net'

if not all([bucket_name, aws_key, aws_secret]):
    print('Ошибка: Не заданы переменные окружения для доступа к S3')
else:
    s3 = boto3.client(
        's3',
        aws_access_key_id=aws_key,
        aws_secret_access_key=aws_secret,
        endpoint_url=endpoint_url,
        config=Config(signature_version='s3v4', s3={'payload_signing_enabled': False})
    )
    try:
        s3.head_bucket(Bucket=bucket_name)
        print("S3 доступен")
    except ClientError as e:
        print("Ошибка подключения к S3:", e)

{'ResponseMetadata': {'RequestId': '81d97a36f9272366',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'nginx',
   'date': 'Tue, 28 Oct 2025 17:05:48 GMT',
   'content-type': 'application/xml',
   'connection': 'keep-alive',
   'keep-alive': 'timeout=60',
   'x-amz-request-id': '81d97a36f9272366'},
  'RetryAttempts': 0}}

S3 доступен


In [32]:
def s3_upload(files: list, s3_folder: str):
    load_dotenv()

    bucket_name = os.getenv('S3_BUCKET_NAME')
    aws_key = os.getenv('AWS_ACCESS_KEY_ID')
    aws_secret = os.getenv('AWS_SECRET_ACCESS_KEY')
    endpoint_url = 'https://storage.yandexcloud.net'
    
    s3 = boto3.client(
        's3',
        aws_access_key_id=aws_key,
        aws_secret_access_key=aws_secret,
        endpoint_url=endpoint_url,
        config=Config(signature_version='s3v4', s3={'payload_signing_enabled': False})
    )
    
    files_to_upload = files
    
    for file_path in files_to_upload:
        basename = os.path.basename(file_path)
        s3_key = f"{s3_folder}{basename}"
        s3.upload_file(file_path, bucket, s3_key)
        print(f"Файл {file_path} успешно загружен в s3://{bucket}/{s3_key}")

In [ ]:
s3_upload(['items.parquet', 'events.parquet'], 'recsys/data/')

# Очистка памяти

Здесь, может понадобится очистка памяти для высвобождения ресурсов для выполнения кода ниже. 

Приведите соответствующие код, комментарии, например:
- код для удаление более ненужных переменных,
- комментарий, что следует перезапустить kernel, выполнить такие-то начальные секции и продолжить с этапа 3.

In [41]:
del tracks
del tracks_v1
del interactions
del interactions_v1

In [ ]:
# для дальнейших этапов рекомендуется перезапустить kernel, выполнить импорты и загрузить файлы .parquet

# === ЭТАП 3 ===

# Загрузка данных

Если необходимо, то загружаем items.parquet, events.parquet.

#### Loading Point

In [7]:
items = pd.read_parquet('items.parquet', engine='pyarrow')
items.head()
events = pd.read_parquet('events.parquet', engine='pyarrow')
events.head()

events_train = pd.read_pickle('events_train.pkl')
events_test = pd.read_pickle('events_test.pkl')
users_train = pd.read_pickle('users_train.pkl')
users_test = pd.read_pickle('users_test.pkl')
catalog_names = pd.read_parquet('catalog_names.parquet')

top_k_pop_items = pd.read_pickle('top_k_pop_items.pkl')

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 24...",[84],[11]


,user_id,track_id,track_seq,started_at
0,1000000,99262,1,2022-07-17
1,1000000,589498,2,2022-07-19
2,1000000,590262,3,2022-07-21
3,1000000,590303,4,2022-07-22
4,1000000,590692,5,2022-07-22


# Разбиение данных

Разбиваем данные на тренировочную, тестовую выборки.

In [45]:
events.head()

,user_id,track_id,track_seq,started_at
0,1000000,99262,1,2022-07-17
1,1000000,589498,2,2022-07-19
2,1000000,590262,3,2022-07-21
3,1000000,590303,4,2022-07-22
4,1000000,590692,5,2022-07-22


In [8]:
%%time
# в тренировочную отнесите все данные до 16 декабря 2022 года (не включительно).
train_test_global_time_split_date = pd.to_datetime("2022-12-16")

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = (list(set(users_train) & set(users_test)))
# количество холодных юзеров
cold_users = list(set(users_test) - set(users_train))

print(len(users_train), len(users_test), len(common_users),len(cold_users))

1342566 783525 752870 30655
CPU times: user 4.9 s, sys: 4.55 s, total: 9.45 s
Wall time: 11.5 s


In [12]:
events_train.to_pickle('events_train.pkl')
events_test.to_pickle('events_test.pkl')
users_train.to_pickle('users_train.pkl')
users_test.to_pickle('users_test.pkl')
catalog_names.to_pickle('catalog_names.pkl')

#### Loading Point

In [ ]:
items = pd.read_parquet('items.parquet', engine='pyarrow')
items.head()
events = pd.read_parquet('events.parquet', engine='pyarrow')
events.head()

events_train = pd.read_pickle('events_train.pkl')
events_test = pd.read_pickle('events_test.pkl')
users_train = pd.read_pickle('users_train.pkl')
users_test = pd.read_pickle('users_test.pkl')
catalog_names = pd.read_parquet('catalog_names.parquet')

top_k_pop_items = pd.read_pickle('top_k_pop_items.pkl')

# Топ популярных

Рассчитаем рекомендации как топ популярных.

In [50]:
events_train.head()

,user_id,track_id,track_seq,started_at
0,1000000,99262,1,2022-07-17
1,1000000,589498,2,2022-07-19
2,1000000,590262,3,2022-07-21
3,1000000,590303,4,2022-07-22
4,1000000,590692,5,2022-07-22


In [10]:
%%time
top_pop_start_date = pd.to_datetime("2022-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["track_id"]).agg(listeners=("user_id", "nunique")).reset_index()

scaler = MinMaxScaler()
item_popularity[["popularity_score"]] = scaler.fit_transform(
    item_popularity[["listeners"]]
)

item_popularity = item_popularity.sort_values(by='popularity_score',ascending=False)

top_k_pop_items = item_popularity.iloc[:100]
top_k_pop_items

CPU times: user 55.2 s, sys: 8.99 s, total: 1min 4s
Wall time: 1min 7s


,track_id,listeners,popularity_score
9098,53404,110026,1.000000
483876,33311009,101076,0.918655
26665,178529,100866,0.916746
512157,35505245,95523,0.868185
368072,24692821,84153,0.764844
...,...,...,...
849893,68562711,47588,0.432511
904051,75944934,47536,0.432038
902657,75642961,47438,0.431147
423054,29175370,47320,0.430075


In [13]:
top_k_pop_items.to_pickle('top_k_pop_items.pkl')
item_popularity.to_pickle('item_popularity.pkl')

#### Loading Point

In [14]:
items = pd.read_parquet('items.parquet', engine='pyarrow')
items.head()
events = pd.read_parquet('events.parquet', engine='pyarrow')
events.head()

events_train = pd.read_pickle('events_train.pkl')
events_test = pd.read_pickle('events_test.pkl')
users_train = pd.read_pickle('users_train.pkl')
users_test = pd.read_pickle('users_test.pkl')
catalog_names = pd.read_parquet('catalog_names.parquet')

top_k_pop_items = pd.read_pickle('top_k_pop_items.pkl')
item_popularity = pd.read_pickle('item_popularity.pkl')

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 24...",[84],[11]


,user_id,track_id,track_seq,started_at
0,1000000,99262,1,2022-07-17
1,1000000,589498,2,2022-07-19
2,1000000,590262,3,2022-07-21
3,1000000,590303,4,2022-07-22
4,1000000,590692,5,2022-07-22


In [7]:
top_k_pop_items\
.merge(catalog_names.query('type=="track"'), left_on='track_id', right_on='id').drop(columns=['type','id']).rename(columns={'name':'track'})\
.merge(items.explode('artists')[['track_id','artists']].drop_duplicates()\
.merge(catalog_names.query('type=="artist"'), left_on='artists', right_on='id').drop(columns=['type','id','artists']),on='track_id')\
.rename(columns={'name':'artist'})

,track_id,listeners,popularity_score,track,artist
0,53404,110026,1.000000,Smells Like Teen Spirit,Nirvana
1,33311009,101076,0.918655,Believer,Imagine Dragons
2,178529,100866,0.916746,Numb,Linkin Park
3,35505245,95523,0.868185,I Got Love,Рем Дигга
4,35505245,95523,0.868185,I Got Love,Miyagi & Эндшпиль
...,...,...,...,...,...
116,75944934,47536,0.432038,Astronaut In The Ocean,Masked Wolf
117,75642961,47438,0.431147,Венера-Юпитер,Ваня Дмитриенко
118,29175370,47320,0.430075,Рапапам,9 грамм
119,29175370,47320,0.430075,Рапапам,Miyagi & Эндшпиль


In [18]:
cold_users_events_with_recs = \
events_test[events_test["user_id"].isin(cold_users)].merge(top_k_pop_items, on="track_id", how="left")
cold_users_events_with_recs=\
cold_users_events_with_recs\
.merge(cold_users_events_with_recs.groupby('track_id')['user_id'].count().reset_index().rename(columns={'user_id':'listeners_fact'}))
cold_users_events_with_recs[["popularity_score_fact"]] = scaler.fit_transform(
    cold_users_events_with_recs[["listeners_fact"]]
)
cold_users_events_with_recs.sort_values(by='listeners')

,user_id,track_id,track_seq,started_at,listeners,popularity_score,listeners_fact,popularity_score_fact
276437,1324025,68348389,60,2022-12-21,47225.0,0.429212,461,0.401747
276647,1891531,68348389,38,2022-12-27,47225.0,0.429212,461,0.401747
276646,1887774,68348389,11,2022-12-29,47225.0,0.429212,461,0.401747
276645,1886297,68348389,23,2022-12-20,47225.0,0.429212,461,0.401747
276644,1884325,68348389,40,2022-12-25,47225.0,0.429212,461,0.401747
...,...,...,...,...,...,...,...,...
1102354,2372855,60342739,23,2022-12-29,NaN,NaN,1,0.000000
1102355,2372943,5029515,1,2022-12-26,NaN,NaN,1,0.000000
1102356,2372943,37563486,3,2022-12-30,NaN,NaN,1,0.000000
1102357,2372960,55293282,24,2022-12-27,NaN,NaN,1,0.000000


In [19]:
cold_users_events_with_recs = \
events_test[events_test["user_id"].isin(cold_users)].merge(top_k_pop_items, on="track_id", how="left")
cold_users_events_with_recs=\
cold_users_events_with_recs\
.merge(cold_users_events_with_recs.groupby('track_id')['user_id'].count().reset_index().rename(columns={'user_id':'listeners_fact'}))
cold_users_events_with_recs[["popularity_score_fact"]] = scaler.fit_transform(
    cold_users_events_with_recs[["listeners_fact"]]
)
cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["popularity_score"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "track_id", "popularity_score", "popularity_score_fact"]]
display(len(cold_users_events_with_recs))
cold_user_recs

1102359

,user_id,track_id,popularity_score,popularity_score_fact
675,1000047,3616433,0.460150,0.336245
676,1006247,3616433,0.460150,0.336245
677,1013479,3616433,0.460150,0.336245
678,1016161,3616433,0.460150,0.336245
679,1019594,3616433,0.460150,0.336245
...,...,...,...,...
351784,2362978,24663745,0.503213,0.479476
351785,2365633,24663745,0.503213,0.479476
351786,2367484,24663745,0.503213,0.479476
351787,2369159,24663745,0.503213,0.479476


In [20]:
from sklearn.metrics import root_mean_squared_error, mean_absolute_error

rmse = root_mean_squared_error(cold_user_recs.drop_duplicates()["popularity_score_fact"], cold_user_recs.drop_duplicates()["popularity_score"])
mae = mean_absolute_error(cold_user_recs.drop_duplicates()["popularity_score_fact"], cold_user_recs.drop_duplicates()["popularity_score"])
print(round(rmse, 2), round(mae, 2))

0.14 0.12


In [ ]:
# Предсказательная сила рекомендаций по топ100 для popularity score слабая

Попробуем оценить рекомендации через метрики классификации (рекомендовано/нерекомендовано)

Определим сколько по медиане слушает треков пользователь соразмерно тестовой выборке

In [82]:
events_test[['user_id','track_id']].groupby('user_id')['track_id'].nunique().median()

6.0

In [8]:
%%time
events_test[['user_id','track_id']].groupby('user_id')['track_id'].nunique().reset_index()\
.rename(columns={'track_id':'tracks'}).groupby('tracks')['user_id'].count().reset_index().rename(columns={'user_id':'users'})\
[['users','tracks']].head(10)

CPU times: user 6.25 s, sys: 192 ms, total: 6.44 s
Wall time: 6.45 s


,users,tracks
0,130513,1
1,90646,2
2,66127,3
3,49515,4
4,38734,5
5,31397,6
6,25759,7
7,22288,8
8,19295,9
9,16832,10


In [94]:
# %%time
pd.qcut(
list(events_test[['user_id','track_id']].groupby('user_id')['track_id'].nunique().reset_index()\
.rename(columns={'track_id':'tracks'}).sort_values(by='tracks')['tracks'])
,10,duplicates='drop')

[(0.999, 2.0], (0.999, 2.0], (0.999, 2.0], (0.999, 2.0], (0.999, 2.0], ..., (51.0, 682.0], (51.0, 682.0], (51.0, 682.0], (51.0, 682.0], (51.0, 682.0]]
Length: 783525
Categories (9, interval[float64, right]): [(0.999, 2.0] < (2.0, 3.0] < (3.0, 4.0] < (4.0, 6.0] ... (9.0, 15.0] < (15.0, 25.0] < (25.0, 51.0] < (51.0, 682.0]]

In [45]:
events_test[['user_id','track_id']].groupby('user_id')['track_id'].nunique().reset_index()\
.rename(columns={'track_id':'tracks'}).query('tracks<=20')['user_id'].nunique()/events_test['user_id'].nunique()

0.7643087329695926

3/4 пользователей слушает 20 треков или меньше

Каждому юзеру рекомендуем 20 треков

In [34]:
top_k_pop_items20 = item_popularity.iloc[:20]
top_k_pop_items20

,track_id,listeners,popularity_score
9098,53404,110026,1.000000
483876,33311009,101076,0.918655
26665,178529,100866,0.916746
512157,35505245,95523,0.868185
368072,24692821,84153,0.764844
90461,795836,83749,0.761172
197265,6705392,80608,0.732624
475289,32947997,80243,0.729307
6558,37384,79512,0.722663
647236,45499814,78564,0.714047


In [35]:
%%time
recommended_top=\
events_test[['user_id']].drop_duplicates()\
.merge(top_k_pop_items20[['track_id']].rename(columns={'track_id':'recommended_track_id'}), how = 'cross')
recommended_top

CPU times: user 342 ms, sys: 192 ms, total: 533 ms
Wall time: 558 ms


,user_id,recommended_track_id
0,1000003,53404
1,1000003,33311009
2,1000003,178529
3,1000003,35505245
4,1000003,24692821
...,...,...
15670495,2373220,328683
15670496,2373220,60292250
15670497,2373220,17079396
15670498,2373220,47627256


In [36]:
%%time
recommended_top_test=\
events_test[['user_id','track_id']].merge(recommended_top, on='user_id', how='left')
recommended_top_test

CPU times: user 6.48 s, sys: 5.45 s, total: 11.9 s
Wall time: 14.7 s


,user_id,track_id,recommended_track_id
0,1000003,78194999,53404
1,1000003,78194999,33311009
2,1000003,78194999,178529
3,1000003,78194999,35505245
4,1000003,78194999,24692821
...,...,...,...
277972915,2373220,100736375,328683
277972916,2373220,100736375,60292250
277972917,2373220,100736375,17079396
277972918,2373220,100736375,47627256


In [37]:
recommended_top_test['is_hit'] = (recommended_top_test['track_id'] == recommended_top_test['recommended_track_id']).astype(int)
accuracy = recommended_top_test['is_hit'].mean()
accuracy

0.0002918197930935143

Достоверность очень низкая

In [46]:
recommended_top.to_parquet('top_popular.parquet')

In [33]:
s3_upload(['top_popular.parquet'], 'recsys/recommendations/')

Файл top_popular.parquet успешно загружен в s3://s3-student-mle-20250619-6c56b203e2-freetrack/recsys/recommendations/top_popular.parquet


# Персональные

Рассчитаем персональные рекомендации.

### Loading Point

In [43]:
items = pd.read_parquet('items.parquet', engine='pyarrow')
items.head()
events = pd.read_parquet('events.parquet', engine='pyarrow')
events.head()

events_train = pd.read_pickle('events_train.pkl')
events_test = pd.read_pickle('events_test.pkl')
users_train = pd.read_pickle('users_train.pkl')
users_test = pd.read_pickle('users_test.pkl')
catalog_names = pd.read_parquet('catalog_names.parquet', engine='pyarrow')

top_k_pop_items = pd.read_pickle('top_k_pop_items.pkl')
item_popularity = pd.read_pickle('item_popularity.pkl')

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 24...",[84],[11]


,user_id,track_id,track_seq,started_at
0,1000000,99262,1,2022-07-17
1,1000000,589498,2,2022-07-19
2,1000000,590262,3,2022-07-21
3,1000000,590303,4,2022-07-22
4,1000000,590692,5,2022-07-22


In [46]:
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events["user_id_enc"] = user_encoder.transform(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(events["track_id"])
events["item_id_enc"] = item_encoder.transform(events["track_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["track_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["track_id"])

LabelEncoder()

LabelEncoder()

In [47]:
events_train.to_pickle('events_train.pkl')
events_test.to_pickle('events_test.pkl')

In [69]:
events_train.head()
events_test.head()

,user_id,track_id,track_seq,started_at,user_id_enc,item_id_enc,user_idx,track_idx
0,1000000,99262,1,2022-07-17,0,16078,0,16078
1,1000000,589498,2,2022-07-19,0,60661,0,60661
2,1000000,590262,3,2022-07-21,0,61135,0,61135
3,1000000,590303,4,2022-07-22,0,61158,0,61158
4,1000000,590692,5,2022-07-22,0,61386,0,61386


,user_id,track_id,track_seq,started_at,user_id_enc,item_id_enc,user_idx,track_idx
32,1000003,78194999,33,2022-12-20,3,915729,3,915729
244,1000004,83375862,245,2022-12-18,4,942162,4,942162
245,1000004,83764704,246,2022-12-18,4,943785,4,943785
246,1000004,83858540,247,2022-12-20,4,944147,4,944147
247,1000004,84099295,248,2022-12-21,4,945046,4,945046


In [51]:
%%time
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train.assign(listened=1)["listened"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [52]:
user_item_matrix_train

<1373221x1000000 sparse matrix of type '<class 'numpy.int8'>'
	with 208731252 stored elements in Compressed Sparse Row format>

In [54]:
%%time
os.environ["OPENBLAS_NUM_THREADS"] = "4"
als_model = AlternatingLeastSquares(
    factors=10,
    iterations=10,
    regularization=0.05,
    random_state=0,
    use_native=True,
    use_cg=True
)
als_model.fit(user_item_matrix_train)

  0%|          | 0/10 [00:00<?, ?it/s]

In [87]:
%%time
user_ids_encoded = range(len(user_encoder.classes_))

als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100)

CPU times: user 1h 48min 17s, sys: 2h 17min 24s, total: 4h 5min 41s
Wall time: 34min 3s


In [88]:
# with open('als_recommendations_raw.pkl', 'wb') as f:
#     pickle.dump(als_recommendations, f)

# with open('als_recommendations_raw.pkl', 'rb') as f:
#     als_recommendations = pickle.load(f)

In [89]:
%%time
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    return recommendations

CPU times: user 10 µs, sys: 149 µs, total: 159 µs
Wall time: 446 µs


In [91]:
%%time
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

CPU times: user 21.4 s, sys: 26.2 s, total: 47.6 s
Wall time: 59.6 s


In [92]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("personal_als.parquet")

In [94]:
als_recommendations

,user_id,item_id,score
0,1000000,178529,0.006575
1,1000000,732401,0.006179
2,1000000,580880,0.005981
3,1000000,560468,0.005858
4,1000000,560473,0.005704
...,...,...,...
137322095,2373220,73538338,0.067066
137322096,2373220,64351321,0.066999
137322097,2373220,53117514,0.066785
137322098,2373220,85660864,0.066690


# Похожие

Рассчитаем похожие, они позже пригодятся для онлайн-рекомендаций.

### Loading point

In [2]:
items = pd.read_parquet('items.parquet', engine='pyarrow')
items.head()
events = pd.read_parquet('events.parquet', engine='pyarrow')
events.head()
als_recommendations = pd.read_parquet('als_recommendations.parquet', engine='pyarrow')

events_train = pd.read_pickle('events_train.pkl')
events_test = pd.read_pickle('events_test.pkl')
users_train = pd.read_pickle('users_train.pkl')
users_test = pd.read_pickle('users_test.pkl')
catalog_names = pd.read_parquet('catalog_names.parquet', engine='pyarrow')

top_k_pop_items = pd.read_pickle('top_k_pop_items.pkl')
item_popularity = pd.read_pickle('item_popularity.pkl')
als_model = pd.read_pickle('als_model.pkl')

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 24...",[84],[11]


,user_id,track_id,track_seq,started_at
0,1000000,99262,1,2022-07-17
1,1000000,589498,2,2022-07-19
2,1000000,590262,3,2022-07-21
3,1000000,590303,4,2022-07-22
4,1000000,590692,5,2022-07-22


,user_id,item_id,score
0,1000000,178529,0.006575
1,1000000,732401,0.006179
2,1000000,580880,0.005981
3,1000000,560468,0.005858
4,1000000,560473,0.005704


In [20]:
%%time
# events_train = pd.read_pickle('events_train.pkl')

events_train['user_idx'], user_labels = pd.factorize(events_train['user_id'])
events_train['item_idx'], item_labels = pd.factorize(events_train['track_id'])
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train.assign(listened=1)["listened"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

CPU times: user 5.34 s, sys: 11.8 s, total: 17.2 s
Wall time: 45.8 s


In [11]:
%%time
os.environ["OPENBLAS_NUM_THREADS"] = "4"
als_model = AlternatingLeastSquares(factors=10, regularization=0.01, iterations=10)
als_model.fit(user_item_matrix_train.T)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed csc_matrix instead. Converting to CSR took 1.6340327262878418 seconds
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?it/s]

CPU times: user 12min 24s, sys: 3.77 s, total: 12min 28s
Wall time: 4min 12s


In [12]:
# with open('als_model.pkl', 'wb') as f:
#     pickle.dump(als_model, f)

In [16]:
items = pd.read_parquet('items.parquet', engine='pyarrow')
items.head()

,track_id,albums,artists,genres
0,26,"[3, 2490753]",[16],"[11, 21]"
1,38,"[3, 2490753]",[16],"[11, 21]"
2,135,"[12, 214, 2490809]",[84],[11]
3,136,"[12, 214, 2490809]",[84],[11]
4,138,"[12, 214, 322, 72275, 72292, 91199, 213505, 24...",[84],[11]


In [54]:
items.tail()

,track_id,albums,artists,genres
999995,101478482,[21399811],[5540395],"[3, 75]"
999996,101490148,[21403052],[9078726],"[11, 20]"
999997,101493057,[21403883],[11865715],"[44, 75]"
999998,101495927,[21404975],[4462686],"[3, 75]"
999999,101521819,[21414638],[5056591],"[3, 75]"


In [62]:
items.track_id.nunique()

980977

In [56]:
item_id = 101521819

itemidx_map = dict(zip(item_labels, range(len(item_labels))))
target_item_idx = itemidx_map[item_id]

N = 10
sim_items = als_model.similar_items(target_item_idx, N=N+1)

sim_items = [x for x in sim_items if x[0] != target_item_idx]

itemidx_inv = dict(enumerate(item_labels))
similar_track_ids = [itemidx_inv[int(pair[0])] for pair in sim_items]

print(f"Tracks similar to {item_id}: {similar_track_ids}")

Tracks similar to 101521819: [589498]


In [39]:
batch_size = 10000
N = 11
total = len(item_labels)
itemidx_inv = dict(enumerate(item_labels))

for batch_start in range(0, total, batch_size):
    batch_end = min(batch_start + batch_size, total)
    backup_fname = f'i2ibatches/batch_item_idx_from_{batch_start}_to_{batch_end-1}.pkl'

    if os.path.exists(backup_fname):
        print(f"Batch {backup_fname} already exists - skipping")
        continue

    batch_item_idxs = range(batch_start, batch_end)
    batch_item_ids = item_labels[batch_start:batch_end]
    batch_dict = {}

    for item_idx, item_id in zip(batch_item_idxs, batch_item_ids):
        sim_items = als.similar_items(item_idx, N=N)
        recs = []
        for pair in sim_items:
            idx = int(pair[0])
            if idx == item_idx:
                continue
            if idx in itemidx_inv:
                recs.append(itemidx_inv[idx])
            if len(recs) == 10:
                break
        batch_dict[item_id] = recs

    with open(backup_fname, 'wb') as f:
        pickle.dump(batch_dict, f)

    percent = 100 * (batch_end) / total
    print(f"Saved {backup_fname} (Progress: {batch_end}/{total} items, {percent:.1f}% complete)")

Saved i2ibatches/batch_item_idx_from_0_to_9999.pkl (Progress: 10000/999695 items, 1.0% complete)
Saved i2ibatches/batch_item_idx_from_10000_to_19999.pkl (Progress: 20000/999695 items, 2.0% complete)
Saved i2ibatches/batch_item_idx_from_20000_to_29999.pkl (Progress: 30000/999695 items, 3.0% complete)
Saved i2ibatches/batch_item_idx_from_30000_to_39999.pkl (Progress: 40000/999695 items, 4.0% complete)
Saved i2ibatches/batch_item_idx_from_40000_to_49999.pkl (Progress: 50000/999695 items, 5.0% complete)
Saved i2ibatches/batch_item_idx_from_50000_to_59999.pkl (Progress: 60000/999695 items, 6.0% complete)
Saved i2ibatches/batch_item_idx_from_60000_to_69999.pkl (Progress: 70000/999695 items, 7.0% complete)
Saved i2ibatches/batch_item_idx_from_70000_to_79999.pkl (Progress: 80000/999695 items, 8.0% complete)
Saved i2ibatches/batch_item_idx_from_80000_to_89999.pkl (Progress: 90000/999695 items, 9.0% complete)
Saved i2ibatches/batch_item_idx_from_90000_to_99999.pkl (Progress: 100000/999695 items,

In [59]:
%%time
import glob, pickle
item2item_recs = {}
for fname in sorted(glob.glob('i2ibatches/batch_item_idx_from_*.pkl')):
    with open(fname, 'rb') as f:
        d = pickle.load(f)
        item2item_recs.update(d)

CPU times: user 218 ms, sys: 35.4 ms, total: 253 ms
Wall time: 318 ms


In [60]:
len(item2item_recs)

999695

In [67]:
for k, v in list(item2item_recs.items())[:4]:
    print(k, v)

99262 []
589498 []
590262 [589498]
590303 [589498]


In [69]:
%%time
from tqdm import tqdm

user_item_score_rows = []

for user_id, group in tqdm(events.groupby('user_id'), total=events['user_id'].nunique()):
    user_tracks = group['track_id'].tolist()
    recs = set()
    for t in user_tracks:
        recs.update(item2item_recs.get(t, []))
    recs = [rec for rec in recs if rec not in user_tracks]
    for rec in recs[:10]:
        user_item_score_rows.append({'user_id': user_id, 'track_id': rec, 'score': 1})

similar_recomendations = pd.DataFrame(user_item_score_rows)

100%|█████████████████████████████████████████████████████| 1373221/1373221 [02:21<00:00, 9736.74it/s]


CPU times: user 1min 44s, sys: 14.4 s, total: 1min 58s
Wall time: 2min 26s


In [70]:
similar_recomendations.head()

,user_id,track_id,score
0,1000001,2212520,1
1,1000001,589498,1
2,1000001,99262,1
3,1000002,589498,1
4,1000002,99262,1


In [71]:
similar_recomendations.to_parquet("similar.parquet")

# Построение признаков

Построим три признака, можно больше, для ранжирующей модели.

### Loading Point

In [9]:
# items = pd.read_parquet('items.parquet', engine='pyarrow')
# items.head()
# events = pd.read_parquet('events.parquet', engine='pyarrow')
# events.head()

events_train = pd.read_pickle('events_train.pkl')
# events_test = pd.read_pickle('events_test.pkl')
# users_train = pd.read_pickle('users_train.pkl')
# users_test = pd.read_pickle('users_test.pkl')
# catalog_names = pd.read_parquet('catalog_names.parquet', engine='pyarrow')

# top_k_pop_items = pd.read_pickle('top_k_pop_items.pkl')
# item_popularity = pd.read_pickle('item_popularity.pkl')
# als_model = pd.read_pickle('als_model.pkl')

similar_recomendations = pd.read_parquet('similar.parquet', engine='pyarrow')
als_recommendations = pd.read_parquet('personal_als.parquet', engine='pyarrow')

In [11]:
users_tracks_listened=\
events_train.groupby('user_id')['track_id'].count().reset_index()\
.rename(columns={'track_id':'tracks_listened'})
users_tracks_listened

,user_id,tracks_listened
0,1000000,26
1,1000001,36
2,1000002,14
3,1000003,32
4,1000004,244
...,...,...
1342561,2373216,9
1342562,2373217,24
1342563,2373218,277
1342564,2373219,488


In [12]:
%%time
users_temporal_patterns=\
events_train.groupby('user_id').agg({'started_at':['min','max']}).reset_index()
users_temporal_patterns.columns=\
['_'.join(map(str, col)).rstrip('_') for col in users_temporal_patterns.columns.values]
users_temporal_patterns['first_timepoint'] = events_train.started_at.min()
users_temporal_patterns['lifetime'] = (users_temporal_patterns['started_at_max'] - users_temporal_patterns['started_at_min']).dt.days
users_temporal_patterns['cooldown_days'] = (users_temporal_patterns['started_at_min'] - users_temporal_patterns['first_timepoint']).dt.days
users_temporal_patterns

CPU times: user 2.23 s, sys: 1.3 s, total: 3.52 s
Wall time: 5.63 s


,user_id,started_at_min,started_at_max,first_timepoint,lifetime,cooldown_days
0,1000000,2022-07-17,2022-09-15,2022-01-01,60,197
1,1000001,2022-06-11,2022-07-28,2022-01-01,47,161
2,1000002,2022-04-15,2022-08-30,2022-01-01,137,104
3,1000003,2022-02-05,2022-12-14,2022-01-01,312,35
4,1000004,2022-03-29,2022-12-15,2022-01-01,261,87
...,...,...,...,...,...,...
1342561,2373216,2022-07-18,2022-12-15,2022-01-01,150,198
1342562,2373217,2022-08-21,2022-10-22,2022-01-01,62,232
1342563,2373218,2022-03-10,2022-10-05,2022-01-01,209,68
1342564,2373219,2022-04-13,2022-12-15,2022-01-01,246,102


In [13]:
users_data=\
users_tracks_listened.merge(users_temporal_patterns
                            [['user_id','lifetime','cooldown_days']], on = 'user_id')
scaler = MinMaxScaler()
users_data[["tracks_listened","lifetime","cooldown_days"]] = scaler.fit_transform(
    users_data[["tracks_listened","lifetime","cooldown_days"]]
)
users_data

,user_id,tracks_listened,lifetime,cooldown_days
0,1000000,0.001567,0.172414,0.566092
1,1000001,0.002194,0.135057,0.462644
2,1000002,0.000815,0.393678,0.298851
3,1000003,0.001943,0.896552,0.100575
4,1000004,0.015231,0.750000,0.250000
...,...,...,...,...
1342561,2373216,0.000501,0.431034,0.568966
1342562,2373217,0.001442,0.178161,0.666667
1342563,2373218,0.017300,0.600575,0.195402
1342564,2373219,0.030525,0.706897,0.293103


# Ранжирование рекомендаций

Построим ранжирующую модель, чтобы сделать рекомендации более точными. Отранжируем рекомендации.

In [14]:
similar_recomendations.head()
als_recommendations.head()

,user_id,track_id,score
0,1000001,2212520,1
1,1000001,589498,1
2,1000001,99262,1
3,1000002,589498,1
4,1000002,99262,1


,user_id,item_id,score
0,1000000,178529,0.006575
1,1000000,732401,0.006179
2,1000000,580880,0.005981
3,1000000,560468,0.005858
4,1000000,560473,0.005704


In [16]:
als_recommendations.score.median()

0.03378584608435631

In [17]:
%%time
candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    similar_recomendations.rename(columns={'track_id':'item_id'})[["user_id", "item_id", "score"]].rename(columns={"score": "sml_score"}),
    on=["user_id", "item_id"],
    how="outer").fillna(0)
candidates
candidates.to_parquet("candidates.parquet")

CPU times: user 21.4 s, sys: 29.6 s, total: 51 s
Wall time: 1min 4s


,user_id,item_id,als_score,sml_score
0,1000000,178529,0.006575,0.0
1,1000000,732401,0.006179,0.0
2,1000000,580880,0.005981,0.0
3,1000000,560468,0.005858,0.0
4,1000000,560473,0.005704,0.0
...,...,...,...,...
141981021,2373220,149093,0.000000,1.0
141981022,2373220,2212520,0.000000,1.0
141981023,2373220,589498,0.000000,1.0
141981024,2373220,59775485,0.000000,1.0


In [18]:
scaler = MinMaxScaler()
candidates[["als_score"]] = scaler.fit_transform(
    candidates[["als_score"]]
)
candidates

,user_id,item_id,als_score,sml_score
0,1000000,178529,0.001716,0.0
1,1000000,732401,0.001613,0.0
2,1000000,580880,0.001561,0.0
3,1000000,560468,0.001529,0.0
4,1000000,560473,0.001489,0.0
...,...,...,...,...
141981021,2373220,149093,0.000000,1.0
141981022,2373220,2212520,0.000000,1.0
141981023,2373220,589498,0.000000,1.0
141981024,2373220,59775485,0.000000,1.0


In [15]:
# candidates = pd.read_parquet('candidates.parquet', engine='pyarrow')

In [16]:
candidates_enriched=\
candidates.merge(users_data, on='user_id', how='left')
candidates_enriched

,user_id,item_id,als_score,sml_score,tracks_listened,lifetime,cooldown_days
0,1000000,178529,0.001716,0.0,0.001567,0.172414,0.566092
1,1000000,732401,0.001613,0.0,0.001567,0.172414,0.566092
2,1000000,580880,0.001561,0.0,0.001567,0.172414,0.566092
3,1000000,560468,0.001529,0.0,0.001567,0.172414,0.566092
4,1000000,560473,0.001489,0.0,0.001567,0.172414,0.566092
...,...,...,...,...,...,...,...
141981021,2373220,149093,0.000000,1.0,0.012912,0.109195,0.890805
141981022,2373220,2212520,0.000000,1.0,0.012912,0.109195,0.890805
141981023,2373220,589498,0.000000,1.0,0.012912,0.109195,0.890805
141981024,2373220,59775485,0.000000,1.0,0.012912,0.109195,0.890805


In [17]:
als_score_md = candidates_enriched.als_score.median()
als_score_md

0.008278627338424023

In [18]:
candidates_enriched['target']=\
np.where((candidates_enriched['als_score'] > als_score_md) | (candidates_enriched['sml_score'] > 0), 1, 0)
candidates_enriched

,user_id,item_id,als_score,sml_score,tracks_listened,lifetime,cooldown_days,target
0,1000000,178529,0.001716,0.0,0.001567,0.172414,0.566092,0
1,1000000,732401,0.001613,0.0,0.001567,0.172414,0.566092,0
2,1000000,580880,0.001561,0.0,0.001567,0.172414,0.566092,0
3,1000000,560468,0.001529,0.0,0.001567,0.172414,0.566092,0
4,1000000,560473,0.001489,0.0,0.001567,0.172414,0.566092,0
...,...,...,...,...,...,...,...,...
141981021,2373220,149093,0.000000,1.0,0.012912,0.109195,0.890805,1
141981022,2373220,2212520,0.000000,1.0,0.012912,0.109195,0.890805,1
141981023,2373220,589498,0.000000,1.0,0.012912,0.109195,0.890805,1
141981024,2373220,59775485,0.000000,1.0,0.012912,0.109195,0.890805,1


In [21]:
features = ['als_score', 'sml_score', 
            'tracks_listened','lifetime','cooldown_days']
target = 'target'

In [19]:
%%time
train_data = Pool(
    data=candidates_enriched[features], 
    label=candidates_enriched[target])

cb_model = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    depth=5,
    loss_function='Logloss',
    verbose=100,
    random_seed=0,
)

cb_model.fit(train_data)
cb_model.save_model("cb_model.cbm")

0:	learn: 0.5981824	total: 11.5s	remaining: 18m 55s
99:	learn: 0.0010549	total: 18m 9s	remaining: 0us
CPU times: user 25min 59s, sys: 46min 9s, total: 1h 12min 8s
Wall time: 18min 21s


In [19]:
# cb_model.save_model("cb_model.cbm")
cb_model = CatBoostClassifier()
cb_model.load_model("cb_model.cbm")

In [22]:
inference_data = Pool(data=candidates_enriched[features])
predictions = cb_model.predict_proba(inference_data)

candidates_enriched["cb_score"] = predictions[:, 1]

candidates_enriched = candidates_enriched.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_enriched["rank"] = candidates_enriched.groupby("user_id")["cb_score"].rank(method="first", ascending=False)

max_recommendations_per_user = 100
final_recommendations = candidates_enriched.query("rank <= @max_recommendations_per_user")
len(final_recommendations)
final_recommendations.head()

137322100

,user_id,item_id,als_score,sml_score,tracks_listened,lifetime,cooldown_days,target,cb_score,rank
12,1000000,178495,0.001264,0.0,0.001567,0.172414,0.566092,0,0.000038,1.0
13,1000000,29175370,0.001261,0.0,0.001567,0.172414,0.566092,0,0.000038,2.0
14,1000000,732388,0.001237,0.0,0.001567,0.172414,0.566092,0,0.000038,3.0
15,1000000,24663745,0.001222,0.0,0.001567,0.172414,0.566092,0,0.000038,4.0
16,1000000,37384,0.001220,0.0,0.001567,0.172414,0.566092,0,0.000038,5.0


In [23]:
final_recommendations.to_parquet("final_recommendations.parquet")

# Оценка качества

Проверим оценку качества трёх типов рекомендаций: 

- топ популярных,
- персональных, полученных при помощи ALS,
- итоговых
  
по четырем метрикам: recall, precision, coverage, novelty.

### Loading Point

In [2]:
events_test = pd.read_pickle('events_test.pkl')
top_popular = pd.read_parquet('top_popular.parquet', engine='pyarrow')
personal_als = pd.read_parquet('personal_als.parquet', engine='pyarrow')
similar = pd.read_parquet('similar.parquet', engine='pyarrow')
final_recommendations = pd.read_parquet('final_recommendations.parquet', engine='pyarrow')

In [6]:
import pandas as pd

def evaluate_recs(
    recs: pd.DataFrame, 
    test: pd.DataFrame, 
    user_col='user_id', 
    item_col='track_id', 
    N=10
):
    recs = recs.groupby(user_col).head(N)

    recs_by_user = recs.groupby(user_col)[item_col].apply(set)
    test_by_user = test.groupby(user_col)[item_col].apply(set)
    users = set(recs_by_user.index) & set(test_by_user.index)

    precisions, recalls = [], []
    for user in users:
        recs_set = recs_by_user.get(user, set())
        test_set = test_by_user.get(user, set())
        hits = recs_set & test_set
        if len(recs_set) == 0 or len(test_set) == 0:
            continue
        precisions.append(len(hits) / len(recs_set))
        recalls.append(len(hits) / len(test_set))

    precision_at_n = sum(precisions) / len(precisions) if precisions else 0
    recall_at_n = sum(recalls) / len(recalls) if recalls else 0

    all_items_in_recs = set(recs[item_col])
    all_items_in_test = set(test[item_col])
    coverage = len(all_items_in_recs) / len(all_items_in_test) if all_items_in_test else 0

    pop = test[item_col].value_counts()
    pop_ranks = {t: r for r, (t, _) in enumerate(pop.items(), 1)}
    novelty_list = [pop_ranks.get(t, len(pop) + 1) for t in recs[item_col]]
    novelty = sum(novelty_list) / len(novelty_list) if novelty_list else 0

    return {
        'precision': precision_at_n,
        'recall': recall_at_n,
        'coverage': coverage,
        'novelty': novelty,
    }

In [11]:
%%time
evaluate_recs(
    top_popular.rename(columns={'recommended_track_id':'track_id'}),
    events_test[['user_id','track_id']]
)

{'precision': 0.00378226604128777,
 'recall': 0.002933242255144138,
 'coverage': 1.6825782483014373e-05,
 'novelty': 901.6}

In [15]:
%%time
evaluate_recs(
    personal_als.rename(columns={'item_id':'track_id'})[['user_id','track_id']],
    events_test[['user_id','track_id']]
)

CPU times: user 30.4 s, sys: 4.95 s, total: 35.4 s
Wall time: 37.3 s


{'precision': 0.006782297948374334,
 'recall': 0.009279715585680538,
 'coverage': 0.0010280553097121782,
 'novelty': 8614.325591146655}

In [17]:
%%time
evaluate_recs(
    similar[['user_id','track_id']],
    events_test[['user_id','track_id']]
)

CPU times: user 22.2 s, sys: 740 ms, total: 22.9 s
Wall time: 23.1 s


{'precision': 0.0,
 'recall': 0.0,
 'coverage': 0.004783569959920986,
 'novelty': 259318.6653490534}

In [31]:
%%time
evaluate_recs(
    final_recommendations.query('rank<=6')\
    .rename(columns={'item_id':'track_id'})[['user_id','track_id']],
    events_test[['user_id','track_id']]
)

CPU times: user 24.8 s, sys: 4.68 s, total: 29.5 s
Wall time: 30.1 s


{'precision': 0.003184752666900652,
 'recall': 0.0030518363591342875,
 'coverage': 0.0079602776927141,
 'novelty': 143851.39712313848}

In [35]:
%%time
evaluate_recs(
    final_recommendations.query('rank<=20')\
    .rename(columns={'item_id':'track_id'})[['user_id','track_id']],
    events_test[['user_id','track_id']]
)

CPU times: user 29 s, sys: 5.13 s, total: 34.1 s
Wall time: 34.9 s


{'precision': 0.0044923901598545035,
 'recall': 0.006259668065828585,
 'coverage': 0.008140313565282354,
 'novelty': 93674.10308027623}

### Testing

In [47]:
%%time
evaluate_recs(
    final_recommendations.query('rank.between(10,20)')\
    .rename(columns={'item_id':'track_id'})[['user_id','track_id']],
    events_test[['user_id','track_id']]
)

CPU times: user 29.3 s, sys: 6.14 s, total: 35.4 s
Wall time: 36.5 s


{'precision': 0.0063691649915446225,
 'recall': 0.0077718972620433625,
 'coverage': 0.005527269545670221,
 'novelty': 14212.596637249211}

In [55]:
final_recommendations.query('user_id==1000003 & item_id==78194999')

,user_id,item_id,als_score,sml_score,tracks_listened,lifetime,cooldown_days,target,cb_score,rank


In [52]:
events_test[['user_id','track_id']].query('user_id==1000003')

,user_id,track_id
32,1000003,78194999


In [57]:
%%time
# TODO
os.environ["OPENBLAS_NUM_THREADS"] = "4"
als_model = AlternatingLeastSquares(
    factors=10,
    iterations=10,
    regularization=0.05,
    random_state=0,
    use_native=True,
    use_cg=True
)
als_model.fit(user_item_matrix_train)

# ---/---/---/---/---

os.environ["OPENBLAS_NUM_THREADS"] = "4"
from implicit.als import AlternatingLeastSquares

# Improved parameters
factors = 100
iterations = 20
regularization = 0.1
alpha = 40

als_model = AlternatingLeastSquares(
    factors=factors,
    iterations=iterations,
    regularization=regularization,
    random_state=0,
    use_native=True,
    use_cg=True
)

# Confidence weighting for implicit feedback
als_model.fit(alpha * user_item_matrix_train)

UsageError: Line magic function `%%time` not found.


# === Выводы, метрики ===

Основные выводы при работе над расчётом рекомендаций, рассчитанные метрики.

Предсказательная всех полученных рекомендаций по топ100 для popularity score слабая
* Для модели топ100 получили очень низкую точность 0.003 и чрезвычайно низкое покрытие менее 0.00002, высокий уровень новизны
* Для персональных ALS получили точность в 2 раза выше и лучше покрытие
* Для варианта Similar i2i получили нулевую точность - вероятность в текущем виде алгоритм непригоден
* Для ранжирующего алгоритма получили лучше полноту, но точность выше чем в топ100, но ниже персональных ALS
<br>
<strong> Наиболее удачным сейчас выглядит вариант с ALS с учётом дальнейших экспериментов на допонительные условия и гиперпараметры. </strong>

## To Service

In [5]:
final_recommendations[['user_id','item_id','cb_score','rank']]

,user_id,item_id,cb_score,rank
12,1000000,178495,0.000038,1.0
13,1000000,29175370,0.000038,2.0
14,1000000,732388,0.000038,3.0
15,1000000,24663745,0.000038,4.0
16,1000000,37384,0.000038,5.0
...,...,...,...,...
137322090,2373220,70977298,0.999960,96.0
137322091,2373220,78619463,0.999960,97.0
137322092,2373220,23269133,0.999960,98.0
137322093,2373220,41899516,0.999960,99.0


In [6]:
%%time
final_recommendations[['user_id','item_id','cb_score','rank']].to_pickle('final_recommendations.pickle')

CPU times: user 309 ms, sys: 2.98 s, total: 3.29 s
Wall time: 7.01 s


In [7]:
recommendations = final_recommendations[['user_id','item_id','cb_score','rank']]
recommendations

,user_id,item_id,cb_score,rank
12,1000000,178495,0.000038,1.0
13,1000000,29175370,0.000038,2.0
14,1000000,732388,0.000038,3.0
15,1000000,24663745,0.000038,4.0
16,1000000,37384,0.000038,5.0
...,...,...,...,...
137322090,2373220,70977298,0.999960,96.0
137322091,2373220,78619463,0.999960,97.0
137322092,2373220,23269133,0.999960,98.0
137322093,2373220,41899516,0.999960,99.0


In [8]:
%%time
recommendations.to_parquet("recommendations.parquet")

In [13]:
%%time
import pyarrow.parquet as pq

table = pq.read_table("recommendations.parquet")
chunk_size = 10_000_000
num_chunks = table.num_rows // chunk_size + int(table.num_rows % chunk_size > 0)

for i in range(num_chunks):
    start = i * chunk_size
    end = min((i+1) * chunk_size, table.num_rows)
    chunk = table.slice(start, end - start)
    pq.write_table(chunk, f"recommendations_chunked/recommendations_chunk_{i}.parquet")

CPU times: user 8.35 s, sys: 4.19 s, total: 12.5 s
Wall time: 11.5 s
